In [1]:
## Lab 2: Part 1a

# !pip install requests
import os
import requests
import zipfile
import json

In [3]:
# Step 1: Set working directory
directory = r"C:\Users\lande174\Desktop\GIS5571\Lab2\Data"
os.chdir(directory)
print(os.getcwd())

C:\Users\lande174\Desktop\GIS5571\Lab2\Data


In [4]:
# Step 2: Download the .las data
las_download = requests.get("https://resources.gisdata.mn.gov/pub/data/elevation/lidar/county/ramsey/laz/3542-29-25.laz")
with open ("./ramsey_data.laz",'wb') as file:
    file.write(las_download.content)

In [5]:
# Step 3: Convert the .las file into a .lasd file
arcpy.conversion.ConvertLas(
    "./ramsey_data.laz",
    "./Exports",
    out_las_dataset="./Exports/ramsey.lasd",
    )

<Result '.\\Exports\\ramsey.lasd'>

In [6]:
# Step 4: Convert the .lasd file to a DEM
arcpy.conversion.LasDatasetToRaster(
    "./Exports/ramsey.lasd",
    "./Exports/ramsey_DEM.tif")

<Result '.\\Exports\\ramsey_DEM.tif'>

In [7]:
# Step 5: Convert the .lasd file to a TIN
arcpy.ddd.LasDatasetToTin(
    "./Exports/ramsey.lasd",
    "./Exports/ramsey_TIN",
    "WINDOW_SIZE",
    max_nodes=1.4e7)

<Result '.\\Exports\\ramsey_TIN'>

In [8]:
# Step 6: Export PDFs of DEM and TIN files
aprx = arcpy.mp.ArcGISProject("CURRENT")

DEM_layout = aprx.listLayouts("DEM_layout")[0]
DEM_layout.exportToPDF("./PDFs/DEM_export.pdf", resolution=300)

TIN_layout = aprx.listLayouts("TIN_layout")[0]
TIN_layout.exportToPDF("./PDFs/TIN_export.pdf", resolution=300)

'./PDFs/TIN_export.pdf'

In [9]:
## Lab 2: Part 1b

# Step 1: Download prism data
prism_download = requests.get("https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip")
with open ("./prism.zip", 'wb') as file2:
    file2.write(prism_download.content)

with zipfile.ZipFile("./prism.zip", 'r') as zip_file:
    zip_file.extractall('prism')

In [10]:
# Step 2: Convert prism data into spacetime cube
prism_mosaic = arcpy.management.CreateMosaicDataset(
    in_workspace = "MyProject.gdb",
    in_mosaicdataset_name = "prism_mosaic",
    coordinate_system = arcpy.SpatialReference(3857),
)

prism_raster = arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset = prism_mosaic,
    raster_type = 'Raster Dataset',
    input_path = r"C:\Users\lande174\Desktop\GIS5571\Lab2\Data\prism"
)

In [11]:
# Step 3: Calculate precipiation and time fields
arcpy.management.CalculateField(
    in_table = "prism_mosaic/Footprint",
    field = "Variable",
    expression = '"Precip"',
    expression_type = "PYTHON3",
    code_block = ' ',
    field_type = 'TEXT',
)

arcpy.management.CalculateField(
    in_table = "prism_mosaic/Footprint",
    field = "Timestamp",
    expression = 'DateAdd(Date(1991, 0, 1), $feature.OBJECTID-1, "year")',
    expression_type = "Arcade",
    code_block = ' ',
    field_type = 'Date',
)

<Result 'prism_mosaic\\Footprint'>

In [12]:
# Step 4: Set multidimensional info
arcpy.management.BuildMultidimensionalInfo(
    in_mosaic_dataset = "prism_mosaic",
    variable_field = "Variable",
    dimension_fields = "Timestamp",
)

<Result 'prism_mosaic'>

In [13]:
# Step 5: Create multidimensional raster
multidim_prism = arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster = 'prism_mosaic',
    out_multidimensional_raster_layer = 'multidim_prism',
    variables = ['Precip'],
    dimension_def = 'ALL'
)

In [14]:
# Step 6: Creat spacetime cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster = multidim_prism,
    output_cube = r"C:\Users\lande174\Desktop\GIS5571\Lab2\Data\Exports\spacetime_cube_multidim_raster.nc",
)

<Result 'C:\\Users\\lande174\\Desktop\\GIS5571\\Lab2\\Data\\Exports\\spacetime_cube_multidim_raster.nc'>